In [26]:
import xarray as xr
from ipywidgets import (
    SelectionSlider,
    SelectMultiple,
    VBox,
    Layout
)
from ipyleaflet.velocity import Velocity
from ipyleaflet import Map, basemaps, FullScreenControl,ImageOverlay
from ipywidgets import Play, jslink, HBox, IntSlider
import csv
import pandas as pd

In [28]:
tide_level = xr.open_dataset('../data/intermediate/tide_level_h.nc')
tide_level = tide_level.sel(lat=slice(30, 32.5), lon=slice(None, 123.5))
flood_level = xr.open_dataset('../data/intermediate/water_level_h.nc')
# 读取csv文件
with open('../data/intermediate/boundary_data.csv', mode='r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    for row in csv_reader:
        if row['Boundary'] == 'SW':
            sw_bound = (float(row['Latitude']), float(row['Longitude']))
        elif row['Boundary'] == 'NE':
            ne_bound = (float(row['Latitude']), float(row['Longitude']))
# 设置边界
flood_bounds = [sw_bound, ne_bound]

# 读取csv文件
with open('../data/intermediate/Tide_boundary_data.csv', mode='r') as csv_file_tide:
    csv_reader_tide = csv.DictReader(csv_file_tide)
    for row in csv_reader_tide:
        if row['Boundary'] == 'SW':
            sw_bound_tide = (float(row['Latitude']), float(row['Longitude']))
        elif row['Boundary'] == 'NE':
            ne_bound_tide = (float(row['Latitude']), float(row['Longitude']))
# 设置边界
Tide_bounds = [sw_bound_tide, ne_bound_tide]

u = xr.open_dataset('../data/intermediate/u_h.nc')
# 上海
center_lat = 31.2
center_lon = 121.4

center = [center_lat, center_lon]
zoom = 9

display_options = {
    "velocityType": "Global Wind",
    "displayPosition": "bottomleft",
    "displayEmptyString": "No wind data",
}
map_9711 =Map(
    center=center,
    zoom=zoom,
    interpolation="nearest",
    basemap=basemaps.Gaode.Normal,)

map_9711.layout = Layout(width="1400px", height="900px")
map_9711.add_control(FullScreenControl())

display_options = {
    "velocityType": "Global Wind",
    "displayPosition": "bottomleft",
    "displayEmptyString": "No wind data",
    "speedUnit": "m/s",
    "position": "bottomleft",
    "emptyString": "No wind data",
    "color_scale": [
        "#000033",  # 深蓝
        "#000000"   # 黑色
    ],
    "colorbar_title": "Wind Speed (m/s)"
}


wind_layer = Velocity(
    data=u.isel(time=0),
    zonal_speed="ux",
    meridional_speed="uy",
    latitude_dimension="lat",
    longitude_dimension="lon",
    velocity_scale=0.01,
    max_velocity=20,
    display_options=display_options,
)

# 为图层添加一个名称
wind_layer.name = "Wind Layer"

# 添加图层
map_9711.add_layer(wind_layer)



# 创建一个Play控件
play = Play(
    value=0,
    min=0,
    max=55,
    step=1,
    description="Press play",
    interval=500
)

# 创建一个时间滑动条
slider = IntSlider(
    value=0,
    min=0,
    max=55,
    step=1,
    description='Time:',
    continuous_update=False
)

# 将Play控件和时间滑动条链接起来
jslink((play, 'value'), (slider, 'value'))

# 将播放控件和时间滑动条组合在一起显示
play_control = HBox([play, slider])

water_level_layer = ImageOverlay(url=f'../figures/Water_level/Water_level_0.png',bounds=flood_bounds, opacity=0.9)
tide_layer = ImageOverlay(url=f'../figures/Tide_level/Tide_level_0.png',bounds=Tide_bounds, opacity=0.9)


map_9711.add_layer(water_level_layer)
map_9711.add_layer(tide_layer)

import geopandas as gpd
from ipyleaflet import GeoJSON
import json

with open('../data/raw/shanghai_shapefile/shanghai.geojson','r') as f:
    data = json.load(f)
style = {
    "color": "black",  # 设置边界颜色为蓝色
    "weight": 3,  # 设置边界线宽
    "fillOpacity": 0  # 设置填充透明度
}

geo_json = GeoJSON(data=data, style=style, hover_style={'fillColor': 'red'})
map_9711.add_layer(geo_json)

from ipyleaflet import LegendControl



from ipywidgets import HTML
from ipyleaflet import WidgetControl
import numpy as np

# 提取最大值和最小值
flood_min = round(0.15, 2)
flood_max = round(1.2,2)

tide_min = round(-2, 2)
tide_max = round(2, 2)

# 计算淹没深度和潮位的四分位数值
flood_quartiles = np.linspace(flood_min, flood_max, 4)
tide_quartiles = np.linspace(tide_min, tide_max, 4)

# 定义统一的宽度
legend_width = "120px"

# 创建淹没深度图例
legend_flood = HTML(value=f"""
<div style="font-size:20px; width:{legend_width};">
<b>&nbsp; 淹没深度</b>
<p>&nbsp; <span style="color:#aa0000;">&#9608;</span>&nbsp; {flood_quartiles[3]:.2f} m</p>
<p>&nbsp; <span style="color:#ff5500;">&#9608;</span>&nbsp; {flood_quartiles[2]:.2f} m</p>
<p>&nbsp; <span style="color:#ffaa00;">&#9608;</span>&nbsp; {flood_quartiles[1]:.2f} m</p>
<p>&nbsp; <span style="color:#ffff00;">&#9608;</span>&nbsp; {flood_quartiles[0]:.2f} m</p>
</div>
""")

legend_flood_control = WidgetControl(widget=legend_flood, position='bottomright')
map_9711.add_control(legend_flood_control)

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# 创建颜色映射对象
cmap = plt.get_cmap('Blues')

# 提取四个色彩
colors = [cmap(i) for i in np.linspace(0, 1, 4)][::-1]  # 从白色到深蓝色

# 创建潮位图例
legend_tide = HTML(value=f"""
<div style="font-size:20px; width:{legend_width};">
<b>&nbsp; 潮位</b>
<p>&nbsp; <span style="color:{mcolors.to_hex(colors[0])};">&#9608;</span>&nbsp; {tide_quartiles[3]:.2f} m</p>
<p>&nbsp; <span style="color:{mcolors.to_hex(colors[1])};">&#9608;</span>&nbsp; {tide_quartiles[2]:.2f} m</p>
<p>&nbsp; <span style="color:{mcolors.to_hex(colors[2])};">&#9608;</span>&nbsp; {tide_quartiles[1]:.2f} m</p>
<p>&nbsp; <span style="color:{mcolors.to_hex(colors[3])};">&#9608;</span>&nbsp; {tide_quartiles[0]:.2f} m</p>
</div>
""")

legend_tide_control = WidgetControl(widget=legend_tide, position='bottomright')
map_9711.add_control(legend_tide_control)

from IPython.display import HTML as html_disp
from ipyleaflet import Popup, Marker, MarkerCluster
from IPython.display import display, HTML
from ipywidgets import HTML

from ipyleaflet import Map, Marker, Popup
css = """
<style>
.leaflet-popup-content-wrapper, .leaflet-popup-tip {
    background: none !important;
    border: none !important; 
    box-shadow: none;
}
</style>
"""

display(html_disp(css))

message1 = HTML()
message1.value = f"<style> .big-font {{ font-size: 30px; }} </style><div class='big-font'><b>{str(u.time[0].values).split('.')[0]}</b></div>"

popup = Popup(
    location=(31.9, 121.6),
    child=message1,
    close_button=False,
    auto_close=False,
    close_on_escape_key=False
)

map_9711.add_layer(popup)


# 显示播放控件和地图
display( map_9711,play_control)
# 定义回调函数
def update_wind_layer(change):
    time_index = change['new']
    wind_layer.data = u.isel(time=time_index)
    water_level_layer.url = f'../figures/Water_level/Water_level_{time_index}.png'
    tide_layer.url = f'../figures/Tide_level/Tide_level_{time_index}.png'
    message1.value = f"<style> .big-font {{ font-size: 30px; }} </style><div class='big-font'><b>{str(u.time[time_index].values).split('.')[0]}</b></div>"



# 监听滑块值的变化
slider.observe(update_wind_layer, names='value')

Map(center=[31.2, 121.4], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_ou…

In [3]:
tide_level = xr.open_dataset('../data/intermediate/Tide_level_inter.nc')
u = xr.open_dataset('../data/intermediate/interpolated_data_u.nc')
water_level = xr.open_dataset('../data/intermediate/Water_level_inter.nc')
# 时间从66开始
u = u.sel(time=slice(u.time[66], None,2))
tide_level = tide_level.sel(time=u.time)
water_level = water_level.sel(time=u.time)
u.to_netcdf('../data/intermediate/u_final.nc')
tide_level.to_netcdf('../data/intermediate/tide_level_final.nc')
water_level.to_netcdf('../data/intermediate/water_level_final.nc')

In [25]:
tide_level = xr.open_dataset('../data/intermediate/Tide_level_inter.nc')
u = xr.open_dataset('../data/intermediate/interpolated_data_u.nc')
water_level = xr.open_dataset('../data/intermediate/Water_level_inter.nc')

water_level_h = water_level.sel(time=slice('1997-08-17 23:00:00', None))
tide_level_h = tide_level.sel(time=slice('1997-08-17 23:00:00', None))
u = u.interp(time=pd.date_range(u.time.values[0], u.time.values[-1], freq='1H'))
u_h = u.sel(time=slice('1997-08-17 23:00:00', None))

water_level_h.to_netcdf('../data/intermediate/water_level_h.nc')
tide_level_h.to_netcdf('../data/intermediate/tide_level_h.nc')
u_h.to_netcdf('../data/intermediate/u_h.nc')